## vanillaLSTM2.py as of 11/17/23
> My hand written code with many bugs, using my actual data

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from math import ceil

In [3]:
# Hyperparameters
input_size = 64  # Number of channels
hidden_size = 128  # Size of the hidden state in LSTM
output_size = 2 # x,y vel predictions
num_layers = 2  # Number of LSTM layers
learning_rate = 0.001
num_epochs = 10

mybatchsize = 32

lambdaD = 1e-3
lambdaE = 1e-4
lambdaF = 0.0
starting_update = 10
final_update = 18
cond_num = 1

update_ix = [0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614, 10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432, 20769]


In [4]:
class TimeSeriesLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(TimeSeriesLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        #out = self.fc(out[:, -1, :])  # Take the output of the last time step
        out = self.fc(out)  # Predictions for all time steps
        return out

In [5]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

In [6]:
class CustomTimeSeriesDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        #return (self.data.shape[-1])
        return (self.data.shape[1]) # This only works for 2D inputs I think...

    def __getitem__(self, idx):
        # Assuming 'data' and 'labels' are lists of numpy arrays
        sample_data = self.data[:,idx]
        sample_labels = self.labels[:,idx]
        # You can apply any custom logic here based on the specific requirements of your task
        # Convert to PyTorch tensors
        sample_data = torch.Tensor(sample_data)
        sample_labels = torch.Tensor(sample_labels)
        return sample_data, sample_labels

In [7]:
# Load Data
print("Loading Data")
input_data = None
target_data = None
data_path = r"C:\\Users\\kdmen\\Desktop\\Research\\Data\\Client_Specific_Files"
for i in range(14):
    datafile = "UserID" + str(i) + "_TrainData_8by20770by64.npy"
    full_data = np.load(data_path+"\\"+datafile)
    cond_data = full_data[cond_num-1, update_ix[starting_update]:update_ix[final_update], :]
    data = np.transpose(cond_data)
    if input_data is None:
        input_data = data
    else:
        input_data = np.vstack((input_data, data))

    labelfile = "UserID" + str(i) + "_Labels_8by20770by2.npy"
    full_data = np.load(data_path+"\\"+labelfile)
    cond_data = full_data[cond_num-1, update_ix[starting_update]:update_ix[final_update], :]
    data = np.transpose(cond_data)
    if target_data is None:
        target_data = data
    else:
        target_data = np.vstack((target_data, data))

Loading Data


In [8]:
test_split_idx = ceil(input_data.shape[1]*.8)
testing_inputs = torch.tensor(input_data[:, test_split_idx:], dtype=torch.float)
testing_targets = torch.tensor(target_data[:, test_split_idx:], dtype=torch.float)
training_inputs = torch.tensor(input_data[:, :test_split_idx], dtype=torch.float)
training_targets = torch.tensor(target_data[:, :test_split_idx], dtype=torch.float)
print("Data loaded!")

Data loaded!


In [9]:
# Convert data to DataLoader
print("Create custom datasets")
train_dataset = CustomTimeSeriesDataset(training_inputs, training_targets)
trainloader = DataLoader(train_dataset, batch_size=mybatchsize, shuffle=False)
test_dataset = CustomTimeSeriesDataset(testing_inputs, testing_targets)
testloader = DataLoader(test_dataset, batch_size=mybatchsize, shuffle=False)
print("Datasets and dataloaders created!")

Create custom datasets
Datasets and dataloaders created!


In [39]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming your data is a tensor of shape (14, 64, 7001)
# where (number of users, number of channels, sequence length)
data = torch.randn(896, 7001)

# Reshape your data to (number of users * number of channels, sequence length)
reshaped_data = data.view(7001, -1)

# Create a TensorDataset
dataset = TensorDataset(reshaped_data)

# Batch size
batch_size = 2  # You can adjust this based on your preferences and resources

# Create a DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example of iterating through the DataLoader
for batch in train_loader:
    batch_data = batch[0]
    print("Batch Shape:", batch_data.shape)

Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: torch.Size([2, 896])
Batch Shape: t

Linear Regression Model

In [10]:
# Set random seed for reproducibility
torch.manual_seed(42)
# Initialize the model, loss function, and optimizer
input_size = 20000
output_size = 28
linregr_model = LinearRegressionModel(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(linregr_model.parameters(), lr=0.01)

In [23]:
i=0
for inputs, targets in trainloader:
    i += 1
    #print(f"inputs.shape: {inputs.shape}, inputs.sum(): {inputs.sum()}")
print(i)

219


In [13]:
train_log = []
test_log = []

print_boolean = True
# Training the model
print("Train model")
for epoch in range(num_epochs):
    for inputs, targets in trainloader:

        ##########################################################################################
        # Code which handles sizing...
        total_elements = inputs.numel()
        new_first_dim = total_elements // (mybatchsize * input_size)
        # Calculate the remainder
        remainder = total_elements % (mybatchsize * input_size)
        # Optionally, you can pad or trim the tensor to make it evenly divisible
        if remainder != 0:
            # Pad or trim the tensor to make it evenly divisible
            inputs = inputs.view(-1)[:new_first_dim * mybatchsize * input_size]
        # Create a new variable for the reshaped tensor
        inputs_reshaped = inputs.view(new_first_dim, mybatchsize, input_size)
        
        total_target_elements = targets.numel()
        new_first_dim = total_target_elements // (mybatchsize * input_size)
        # Calculate the remainder
        remainder = total_target_elements % (mybatchsize * input_size)
        # Optionally, you can pad or trim the tensor to make it evenly divisible
        if remainder != 0:
            # Pad or trim the tensor to make it evenly divisible
            targets = targets.view(-1)[:new_first_dim * mybatchsize * 2]
        # Create a new variable for the reshaped tensor
        targets_reshaped = targets.view(new_first_dim, mybatchsize, 2)
        ##########################################################################################

        if epoch==0 and print_boolean==True:
            print_boolean = False
            print("Original size of inputs:", inputs.size())
            print("Reshaped size of inputs:", inputs_reshaped.size())

        optimizer.zero_grad()

        outputs = linregr_model(inputs_reshaped)
        #loss = criterion(outputs, targets)
        t1 = lambdaE*criterion(outputs, targets_reshaped)
        # Initialize a variable to accumulate the norms
        running_weights_norm = 0.0
        # Iterate through the parameters and calculate the norm
        for param in linregr_model.parameters():
            # Uhh make sure this is only norming the weights and not other params (bias or something idk)
            running_weights_norm += torch.norm(param)
        t2 = lambdaD*(running_weights_norm**2)
        t3 = lambdaF*(torch.linalg.matrix_norm(inputs)**2)
        loss = t1 + t2 + t3
        train_log.append(loss.item())
        loss.backward()
        optimizer.step()

    # Test the current model each epoch
    with torch.no_grad():
        for test_inputs, test_targets in testloader:
            linregr_model.eval()

            test_inputs_reshaped = test_inputs.view(-1, mybatchsize, input_size)
            test_targets = test_targets.transpose(0, 1)

            prediction = linregr_model(test_inputs_reshaped)
            t1 = lambdaE*criterion(prediction, test_targets)
            for param in linregr_model.parameters():
                # Uhh make sure this is only norming the weights and not other params (bias or something idk)
                running_weights_norm += torch.norm(param)
            t2 = lambdaD*(running_weights_norm**2)
            t3 = lambdaF*(torch.linalg.matrix_norm(test_inputs)**2)
            test_loss = t1 + t2 + t3
            test_log.append(test_loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
print("Training complete!")

Train model
Original size of inputs: torch.Size([0])
Reshaped size of inputs: torch.Size([0, 32, 20000])


C:\Users\kdmen\miniconda3\envs\fl_torch\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([0, 32, 2])) that is different to the input size (torch.Size([0, 32, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (28) must match the size of tensor b (2) at non-singleton dimension 2

In [14]:
outputs.shape

torch.Size([0, 32, 28])

In [15]:
targets_reshaped.shape

torch.Size([0, 32, 2])

In [16]:
outputs

tensor([], size=(0, 32, 28), grad_fn=<ViewBackward0>)

In [20]:
inputs

tensor([])

In [12]:
# Create LSTM model
print("Create LSTM Model")
model = TimeSeriesLSTM(input_size, hidden_size, output_size, num_layers)
print("Model instantiated!")

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Create LSTM Model
Model instantiated!


New

In [ ]:
total_elements = inputs.numel()
new_first_dim = total_elements // (mybatchsize * input_size)
# Calculate the remainder
remainder = total_elements % (mybatchsize * input_size)
# Optionally, you can pad or trim the tensor to make it evenly divisible
if remainder != 0:
    # Pad or trim the tensor to make it evenly divisible
    inputs = inputs.view(-1)[:new_first_dim * mybatchsize * input_size]
# Create a new variable for the reshaped tensor
inputs_reshaped = inputs.view(new_first_dim, mybatchsize, input_size)

total_target_elements = targets.numel()
new_first_dim = total_target_elements // (mybatchsize * input_size)
# Calculate the remainder
remainder = total_target_elements % (mybatchsize * input_size)
# Optionally, you can pad or trim the tensor to make it evenly divisible
if remainder != 0:
    # Pad or trim the tensor to make it evenly divisible
    targets = targets.view(-1)[:new_first_dim * mybatchsize * 2]
# Create a new variable for the reshaped tensor
targets_reshaped = targets.view(new_first_dim, mybatchsize, 2)

Old

In [ ]:
total_elements = inputs.numel()
new_first_dim = total_elements // (mybatchsize * input_size)
# Calculate the remainder
remainder = total_elements % (mybatchsize * input_size)
# Optionally, you can pad or trim the tensor to make it evenly divisible
if remainder != 0:
    # Pad or trim the tensor to make it evenly divisible
    inputs = inputs.view(-1)[:new_first_dim * mybatchsize * input_size]
# Reshape the tensor
input_reshaped = inputs.view(new_first_dim, mybatchsize, input_size)

total_elements = targets.numel()
new_first_dim = total_elements // (mybatchsize * input_size)
# Calculate the remainder
remainder = total_elements % (mybatchsize * input_size)
# Optionally, you can pad or trim the tensor to make it evenly divisible
if remainder != 0:
    # Pad or trim the tensor to make it evenly divisible
    targets = targets.view(-1)[:new_first_dim * mybatchsize * input_size]
# Reshape the tensor
targets_reshaped = targets.view(new_first_dim, mybatchsize, input_size)

#######################################################
# Reshape to (N, batch_size, input_size) with batch_size = 32
#input_reshaped = inputs.view(-1, mybatchsize, input_size)
#targets_reshaped = targets.transpose(0, 1)  # Didnt fix it... need to have an output size of 2...
#targets_reshaped = targets.view(-1, mybatchsize, 2)
#######################################################

Error
- t1 = lambdaE*criterion(outputs, targets_reshaped)
- RuntimeError: The size of tensor a (2) must match the size of tensor b (64) at non-singleton dimension 2


In [13]:
train_log = []
test_log = []

print_boolean = True
# Training the model
print("Train model")
for epoch in range(num_epochs):
    for inputs, targets in trainloader:
        
        # Code which handles sizing...

        if epoch==0 and print_boolean==True:
            print_boolean = False
            print("Original size of inputs:", inputs.size())
            print("Reshaped size of inputs:", input_reshaped.size())

        ##########################################################################################

        optimizer.zero_grad()

        outputs = model(input_reshaped)
        #loss = criterion(outputs, targets)
        t1 = lambdaE*criterion(outputs, targets_reshaped)
        # Initialize a variable to accumulate the norms
        running_weights_norm = 0.0
        # Iterate through the parameters and calculate the norm
        for param in model.parameters():
            # Uhh make sure this is only norming the weights and not other params (bias or something idk)
            running_weights_norm += torch.norm(param)
        t2 = lambdaD*(running_weights_norm**2)
        t3 = lambdaF*(torch.linalg.matrix_norm(inputs)**2)
        loss = t1 + t2 + t3
        train_log.append(loss.item())
        loss.backward()
        optimizer.step()

    # Test the current model each epoch
    with torch.no_grad():
        for test_inputs, test_targets in testloader:
            model.eval()

            test_inputs_reshaped = test_inputs.view(-1, mybatchsize, input_size)
            test_targets = test_targets.transpose(0, 1)

            prediction = model(test_inputs_reshaped)
            t1 = lambdaE*criterion(prediction, test_targets)
            t2 = lambdaD*(torch.linalg.matrix_norm((model.weight))**2)
            t3 = lambdaF*(torch.linalg.matrix_norm(test_inputs)**2)
            test_loss = t1 + t2 + t3
            test_log.append(test_loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
print("Training complete!")

Train model
Original size of inputs: torch.Size([32, 896])
Reshaped size of inputs: torch.Size([14, 32, 64])


C:\Users\kdmen\miniconda3\envs\fl_torch\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([0, 32, 64])) that is different to the input size (torch.Size([14, 32, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2) must match the size of tensor b (64) at non-singleton dimension 2

In [ ]:
print("Plot training and testing logs!")
plt.plot(range(len(train_log)), train_log, label="Train")
plt.plot(range(len(test_log)), test_log, label="Test")
plt.legend()
plt.title("Loss Per Epoch")
plt.xlabel("Epoch Number")
plt.ylabel("Loss")
plt.show()

In [ ]:
assert(1==0)

## Synthetic/Toy Data LSTM with Variable Training/Inference Sizes

In [36]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Generate synthetic training data with linear relationship
timesteps_train = 896
features_train = 20000
X_train = 5 * np.random.rand(timesteps_train, features_train)
y_train = 2 * np.sum(X_train, axis=1, keepdims=True) + 1 + np.random.randn(timesteps_train, 1)

# Generate synthetic inference data
timesteps_inference = 64
features_inference = 1000
X_inference = 5 * np.random.rand(timesteps_inference, features_inference)
y_inference = 2 * np.sum(X_train, axis=1, keepdims=True) + 1 + np.random.randn(timesteps_train, 1)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).unsqueeze(0)  # Add batch dimension
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(0)  # Add batch dimension
X_inference_tensor = torch.FloatTensor(X_inference).unsqueeze(0)  # Add batch dimension
y_inference_tensor = torch.FloatTensor(y_inference).unsqueeze(0)  # Add batch dimension

# Define LSTM regression model
class LSTMRegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMRegressionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Take the output from the last timestep
        return output

# Define LSTM regression model with dynamic input size
class DynamicInputLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DynamicInputLSTM, self).__init__()
        #self.lstm = nn.LSTMCell(1, hidden_size)  # Input size during training is 20000, set to 1 dynamically during inference
        self.lstm = nn.LSTMCell(features_train, hidden_size)  # Adjust input size during training
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, prev_hidden=None):
        #h_t, c_t = self.lstm(x, prev_hidden)
        #h_t, c_t = self.lstm(x[:, 0, :], prev_hidden)  # Adjust input dimension during training
        #output = self.fc(h_t)
        #return output, (h_t, c_t)

        batch_size, timesteps, input_size = x.size()
        h_t, c_t = self.lstm(x.view(batch_size * timesteps, input_size), prev_hidden)
        output = self.fc(h_t)
        return output, (h_t, c_t)

# Instantiate the model
input_size = features_train
hidden_size = 64
output_size = 1
criterion = nn.MSELoss()

In [28]:
lstm_regr = LSTMRegressionModel(input_size, hidden_size, output_size)
optimizer = optim.SGD(lstm_regr.parameters(), lr=0.01)

# Train the LSTM model
epochs = 30
for epoch in range(epochs):
    lstm_regr.train()
    optimizer.zero_grad()
    y_pred = lstm_regr(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## EVAL IS BROKEN! CANT ACCOMODATE DIFFERENT SIZES...
# Perform inference with the LSTM model
lstm_regr.eval()
with torch.no_grad():
    lstm_inference_predictions = lstm_regr(X_inference_tensor)

# Display the LSTM inference predictions
print("LSTM Inference Predictions:", lstm_inference_predictions.numpy())

Epoch [1/30], Loss: 9997000704.0000
Epoch [11/30], Loss: 6250701.5000
Epoch [21/30], Loss: 172081.0000


In [38]:
lstm_dynamic = DynamicInputLSTM(hidden_size, output_size)
optimizer = optim.SGD(lstm_dynamic.parameters(), lr=0.01)

In [39]:
for epoch in range(epochs):
    lstm_dynamic.train()
    optimizer.zero_grad()
    #for t in range(timesteps_train):
    #    input_t = X_train_tensor[:, t:t+1, :]
    #    output, _ = lstm_dynamic(input_t)
    output, _ = lstm_dynamic(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

C:\Users\kdmen\miniconda3\envs\fl_torch\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1, 896, 1])) that is different to the input size (torch.Size([896, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/30], Loss: 10002085888.0000
Epoch [11/30], Loss: 1261375.1250
Epoch [21/30], Loss: 170105.6719


In [41]:

# Perform inference with the LSTM model
lstm_dynamic.eval()
with torch.no_grad():
    lstm_inference_predictions = []
    prev_hidden = None
    #for t in range(timesteps_inference):
    #    input_t = X_inference_tensor[:, t:t+1, :]
    #    output, prev_hidden = lstm_dynamic(input_t, prev_hidden)
    #    lstm_inference_predictions.append(output.numpy())
    
    #output, _ = lstm_dynamic(X_inference_tensor)
    #lstm_inference_predictions.append(output.numpy())

    for t in range(timesteps_inference):
        input_t = X_inference_tensor[:, t:t+1, :]
        output, _ = lstm_dynamic(input_t)
        lstm_inference_predictions.append(output.numpy())
    
    # Ought to calculate the loss between the inference output and the inference labels...




RuntimeError: input has inconsistent input_size: got 1000 expected 20000